# Diagnosis Pogress
This note book is dedicated to select diabetes patient based on their diagnosis progress data source, the inclusion criterial is based on the corresponding ICD code. This notebook includes following sections:
1. Read the files
2. Find out the first date of diagnosis
3. Write to disk

## Read the files

In [ ]:
import pandas as pd 
import os
import numpy as np
import util.cleaning_tools as tools
import re
from datetime import datetime
%load_ext autoreload
%autoreload 2

In [ ]:
# read the demographic table 
dgp = tools.fileReader(r'../DATAFILE', 'patient_data')
dgp = dgp[['dob_Y','pseudo_patient_key']]

### Read the diagnosis basis defined by file T2DM0615.csv

In [ ]:
# read the diabetes diagnosis ICD9-CM mapping table
diab_mapping = pd.read_csv(r'../tables/input/T2DM0615.csv',header=0)

In [ ]:
#read the diagnosis table mdl_dx_progress.dat.*
dx_progress = tools.fileReader(r'../DATAFILE','mdl_dx_progress')

In [ ]:
patients = pd.read_csv(r'../tables/output/group_patient_age.csv', index_col=0)

In [ ]:
#table diab_patient contains all the diabetes diagnoisis records
diab_records = pd.merge(dx_progress, diab_mapping, how='inner', left_on='term_id', right_on='Term ID')
# subset the table
diab_patients = diab_records[["pseudo_patient_key", "reference_dtm", "diff_in_hour_reference_dtm"]]


### Find out the first date of diagnosis

In [ ]:
first_dx = diab_patients[tools.row_number(diab_patients, "pseudo_patient_key",sort_key="reference_dtm") == 1]

### Write to disk

In [ ]:
#rename to fit the protocol of the fields
first_dx.rename({"reference_dtm":"dx_dtm", "diff_in_hour_reference_dtm": "diff_hour"}, axis='columns', inplace=True)
# add the source
first_dx["src"] = "dx"
# add the diabetes type
diab_patients["diab_type"] = "diab"
# write to disk
first_dx.to_csv(r"../tables/output/first_diag_dx.csv")